In [1]:
import torch
import MiniFL as mfl

In [2]:
def run_client(i:int, client: mfl.algorithms.gd.GDClient):
    client.prepare()
    for _ in range(100):
        _ = client.step()
        
def run_master(master: mfl.algorithms.gd.GDMaster):
    master.prepare()
    for i in range(100):
        print(f"Master: {master.step()}")

In [6]:
from sklearn.datasets import load_svmlight_file

NUM_CLIENTS = 20

data, labels = load_svmlight_file("phishing.txt")
enc_labels = labels.copy()
data_dense = data.todense()


eval_data = (torch.from_numpy(data_dense).to(torch.float32), torch.from_numpy(enc_labels).to(torch.float32)[:, None])
partition_size = (len(eval_data[0]) - 1) // NUM_CLIENTS + 1
clients_data = [(x, y) for x, y in zip(torch.split(eval_data[0], partition_size, dim=0), torch.split(eval_data[1], partition_size, dim=0))]

model = torch.nn.Linear(eval_data[0].shape[1], 1, bias=False)
loss_fn = torch.nn.BCEWithLogitsLoss()

master, clients = mfl.algorithms.get_permk_marina_master_and_clients(
    gamma=100,
    clients_data=clients_data,
    eval_data=eval_data,
    model=model,
    loss_fn=loss_fn,
    p=0.01,
)

In [7]:
import threading

client_threads = []
for i, client in enumerate(clients):
    client_threads.append(threading.Thread(target=run_client, args=(i, client)))
    client_threads[-1].start()
    
master_thread = threading.Thread(target=run_master, args=(master,))
master_thread.start()

master_thread.join()
for t in client_threads:
    t.join()
    


Master: 0.659458577632904
Master: 0.6686608791351318
Master: 0.6666728854179382
Master: 0.669396698474884
Master: 0.6685185432434082
Master: 0.6677916646003723
Master: 0.6662203669548035
Master: 0.6665048599243164
Master: 0.6651984453201294
Master: 0.6652677655220032
Master: 0.6653119325637817
Master: 0.6653526425361633
Master: 0.6653500199317932
Master: 0.6653265953063965
Master: 0.6653242707252502
Master: 0.6653265953063965
Master: 0.6653271913528442
Master: 0.6653284430503845
Master: 0.6653282642364502
Master: 0.6653283834457397
Master: 0.6653283834457397
Master: 0.6653284430503845
Master: 0.6653281450271606
Master: 0.6653279066085815
Master: 0.6653276681900024
Master: 0.6653273701667786
Master: 0.6653277277946472
Master: 0.6653278470039368
Master: 0.6653277277946472
Master: 0.6653277277946472
Master: 0.665327787399292
Master: 0.6653278470039368
Master: 0.6653278470039368
Master: 0.6653278470039368
Master: 0.6653278470039368
Master: 0.6653278470039368
Master: 0.6653278470039368
Mast

In [8]:
clients[0].data_sender.n_bits_passed

16960